# Patent Breakthrough walkthrough

This notebook illustrates the complete analysis process of breakthrough patents, from preparing input files
to calculating impact and novelty scores.

## 1. Preparing input files

There are three input files: a file with patent texts, a patent/year-index, and list of patent/CPC-codes.

### Patents
In its raw format, the input file contains the text of one patent file per line.
Each line starts with a path pointing to that patent's original text 
file (`/Volumes/External/txt/0000000-0100000/US1009.txt`), followed by the patent text. Example file: `./data/raw_input.txt`. 


### Patent/Year-index
Contains the year of publication of each patent. Example file: `./data/year.csv`. 


### CPC-file
The CPC-file (Cooperative Patent Classification) contains the patent classification code for each patent. These codes are used to calculate benchmark similarities. Example file: `./data/GPCPCs.txt`

Note: the included data files only contain a small subset of the original data, for example purposes.

#### Other files
The three other files in the data folder - `greek.txt`  `stopwords.txt`, and `symbols.txt` - are required by the `OldPreprocessor`-class.

In [1]:
from pathlib import Path

data_path = "./data/USPTO"
input_file = Path(f"{data_path}/brief_text")
year_file = Path(f"{data_path}/input_files/year.csv")
cpc_fp = Path(f"{data_path}/input_files/GPCPCs.txt")
patent_dir = Path("./patents")
output_folder = Path("./output")
output_fp = Path("./output", "patents.h5")
results_fp = Path("./results")

output_folder.mkdir(exist_ok=True)
patent_dir.mkdir(exist_ok=True)

### 1.1. Compressing

The compressor function transforms the patents to a more manageable format, sorts and saves them by year of publication, and compresses the resulting files.

In [2]:
import json
import lzma
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import List, Union, Dict

In [23]:
import json
import lzma
import csv
from typing import List, Union, Dict
from tqdm import tqdm
from pathlib import Path
import os
import glob

def read_xz(compressed_fp: Union[Path, str]) -> List[Dict]:
    """Read an .xz file containing patents

    Arguments
    ---------
    compressed_fp:
        File to read the patents from.

    Results
    -------
    patents: List[Dict]
        Patents in the file.
    """
    with lzma.open(compressed_fp, mode="rb") as handle:
        patents = json.loads(handle.read().decode(encoding="utf-8"))
    return patents

def write_xz(compressed_fp: Union[Path, str], patents: List[Dict]) -> None:
    """Write a set of patents to a compressed file

    Arguments
    ---------
    fp:
        File to write to.
    patents:
        Patents to store.
    """
    with lzma.open(compressed_fp, mode="wb", preset=9) as handle:
        handle.write(str.encode(json.dumps(patents), encoding="utf-8"))
    
#     # Get the total number of patents for progress tracking
#     total_patents = len(patents)

#     # Open the compressed file for writing
#     with lzma.open(compressed_fp, mode="wb", preset=9) as handle:
#         # Use tqdm to create a progress bar
#         with tqdm(total=total_patents, desc="Writing patents to compressed file", unit="patent") as pbar:
#             # Iterate through patents and write to the compressed file
#             for patent in patents:
#                 handle.write(str.encode(json.dumps(patent), encoding="utf-8"))
#                 pbar.update(1)  # Update the progress bar

import csv
from typing import List, Dict
from tqdm import tqdm

def parse_patent_file(patent_input_fp: str, year_lookup: str) -> List[Dict[str, str]]:
    """
    Parse a raw patent file into a structured list.

    Parameters:
    -----------
    patent_input_fp : str
        Path to the .tsv input file to process with columns 'patent_id' and 'summary_text', one patent per row.

    year_lookup : str
        Path to the .csv file to lookup the year for each patent ID with columns 'pat' (patient_id) and 'year'.

    Returns:
    --------
    List[Dict[str, str]]
        A sorted list of dictionaries, where each item is for one patent and
        the year of publication:
        {
            'patent': patent_id,
            'file': file name,
            'contents':  summary_text,
            'year': year
        }
    """
    
    print(f"Starting to parse file {patent_input_fp}")
    
    # Increase the field size limit
    csv.field_size_limit(10**8)
    
    # Initialize an empty list to store the structured patent data
    patents = []

    # Read the year lookup data into a dictionary for quick access
    year_dict = {}
    with open(year_lookup, 'r', newline='') as year_file:
        year_reader = csv.DictReader(year_file, delimiter='\t')
        for row in year_reader:
            year_dict[row['pat']] = row['year']  # Convert 'year' to integer

    # Read the raw patent file and parse the data
    with open(patent_input_fp, 'r', newline='') as patent_file:
        patent_reader = csv.DictReader(patent_file, delimiter='\t')
        for row in tqdm(patent_reader):
            patent_id = row['patent_id']
            
            # Skip entries where "patent_id" is not an integer
            if not patent_id.isdigit():
                continue

            summary_text = row['summary_text']
            year = year_dict.get(patent_id, None)

            # Create a dictionary for the current patent
            patent_data = {
                'patent': int(patent_id),  # Convert 'patent_id' to integer
                'file': str(patent_input_fp),
                'contents': summary_text,
                'year': int(year)
            }

            # Append the patent data to the list
            patents.append(patent_data)
    
    patents = sorted(patents, key=lambda x: x["patent"])
    
    return patents


def compress_raw_dir(patent_input_dir: Union[Path, str], year_fp: Union[Path, str], output_dir: Union[Path, str]) -> None:
    """Compress all raw files in a directory.

    For efficiency, it stores which files have already been processed in
    a file called 'processed_files.txt' in the output directory.
    If somehow there is corruption, or re-runs are required, simply
    delete this file.

    This function is not thread-safe.

    Arguments
    ---------
    patent_input_dir:
        Directory containing all raw files with patents.
    year_fp:
        CSV file with publication year for each patent.
    output_dir:
        Directory to write the compressed files to.
    """
    patent_input_dir = Path(patent_input_dir)
    
    for file in patent_input_dir.glob("g_brf_sum_text_*.tsv"):
        
        # Extract year from the filename
        year = str(file).split('_')[-1].split('.')[0]
        
        # Get next file path to process and compress
        patent_input_fp = file
        compressed_fp = output_dir / Path(str(year) + ".xz")
        
        # Check if the output file already exists
        if os.path.exists(compressed_fp):
            print(f"Output file {compressed_fp} already exists. Skipping to the next input file.")
            continue

        # Parse patents for a year in the right format
        patents = parse_patent_file(patent_input_fp, year_fp)

        # Write the patents to a files, numbered by year
        write_xz(compressed_fp, patents)
    
    

# Example usage:
data_path = "./data/USPTO"
# input_file = Path(f"{data_path}/brief_text")
input_file = Path(f"{data_path}/mock")
year_file = Path(f"{data_path}/input_files/year.csv")
patent_dir = Path("./patents")

# Parse and compress files:
compress_raw_dir(input_file, year_file, patent_dir)

# Now 'parsed_patents' contains the structured list of patents

Starting to parse file data/USPTO/mock/g_brf_sum_text_1976.tsv


1000it [00:00, 29233.90it/s]


Starting to parse file data/USPTO/mock/g_brf_sum_text_1977.tsv


1000it [00:00, 26973.71it/s]


Starting to parse file data/USPTO/mock/g_brf_sum_text_1980.tsv


...

1000it [00:00, 24865.01it/s]


Starting to parse file data/USPTO/mock/g_brf_sum_text_1979.tsv


1000it [00:00, 27788.63it/s]


Starting to parse file data/USPTO/mock/g_brf_sum_text_1978.tsv


1000it [00:00, 28484.24it/s]


You now have XZ-compressed files containing patents per year. Each file contains a list of JSON-objects, each JSON-object has the following key/values:

- `patent`: patent's ID
- `file`: path of original text file (not actually used)
- `contents`: patent text
- `year`: year of publication

## 2. Calculating embeddings

We calculate embeddings and scores with four different models: Countvec, Tf-Idf, Doc2Vec, and BERT ([PatentSBERTa](https://github.com/AI-Growth-Lab/PatentSBERTa)).


### 2.1. Preprocessors & parameters
Each model has its own preprocessor with various parameters. Most models also have configurable hyperparameters. The values for these parameters have been optimised using the original dataset, resulting in the values used in the `compute_embeddings()`-function below.

To recalibrate preprocessor and model parameters, run each model's hyperopt-script. See the [readme](https://github.com/UtrechtUniversity/patent-breakthrough/blob/main/docs/hyperparameter.md) and [hyperopt-notebooks](hyperopt/) for more details.


### 2.2. Calculating embeddings
Next, we calculate the embeddings.

In [1]:
from pathlib import Path

data_path = "./data/USPTO"
input_file = Path(f"{data_path}/brief_text")
year_file = Path(f"{data_path}/input_files/year.csv")
cpc_fp = Path(f"{data_path}/input_files/GPCPCs.txt")
patent_dir = Path("./patents")
output_folder = Path("./output")
output_fp = Path("./output", "patents.h5")
results_fp = Path("./results")

output_folder.mkdir(exist_ok=True)
patent_dir.mkdir(exist_ok=True)

In [2]:
import json
import lzma
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import List, Union, Dict

from docembedder.models import TfidfEmbedder
from docembedder.preprocessor.preprocessor import Preprocessor
from docembedder.preprocessor.oldprep import OldPreprocessor
from docembedder.models.doc2vec import D2VEmbedder
# from docembedder.models import CountVecEmbedder
# from docembedder.models import BERTEmbedder

from docembedder.utils import run_models
from docembedder.pretrained_run import pretrained_run_models
import datetime

def check_files(sim_spec):
    for year in range(sim_spec.year_start, sim_spec.year_end):
        if not (patent_dir / f"{year}.xz").is_file():
            raise ValueError(f"Please download patent file {year}.xz and put it in"
                             f"the right directory ({patent_dir})")

# def compute_embeddings_cv(patent_dir, output_fp, cpc_fp, sim_spec, n_jobs):

#     model_cv = {
#         "countvec": CountVecEmbedder(method='sigmoid')
#     }
#     prep_cv = {
#         "prep-countvec": OldPreprocessor(list_path=data_path)
#     }

#     check_files(sim_spec)
#     run_models(prep_cv, model_cv, sim_spec, patent_dir, output_fp, cpc_fp, n_jobs=n_jobs)
#     print('Calculated countvec emdeddings')

    
def compute_embeddings_tfidf(patent_dir, output_fp, cpc_fp, sim_spec, n_jobs):
    
    model_tfidf = {
        "tfidf": TfidfEmbedder(
            ngram_max=1,stop_words='english',stem=False, norm='l1', sublinear_tf=True, min_df=6, max_df=0.665461)
    }
    prep_tfidf = {
        "prep-tfidf": Preprocessor(keep_caps=True, keep_start_section=True, remove_non_alpha=True),
    }

    check_files(sim_spec)
    run_models(prep_tfidf, model_tfidf, sim_spec, patent_dir, output_fp, cpc_fp, n_jobs=n_jobs)
    print('Calculated tfidf emdeddings')

    
# def compute_embeddings_doc2vec(patent_dir, output_fp, cpc_fp, sim_spec, n_jobs):

#     model_doc2vec = {
#         "doc2vec": D2VEmbedder(epoch=8, min_count=13, vector_size=100)
#     }
#     prep_doc2vec = {
#         "prep-doc2vec": Preprocessor(keep_caps=False, keep_start_section=True, remove_non_alpha=False)
#     }

#     check_files(sim_spec)
#     run_models(prep_doc2vec, model_doc2vec, sim_spec, patent_dir, output_fp, cpc_fp, n_jobs=n_jobs)
#     print('Calculated doc2vec emdeddings')

# def compute_embeddings_bert(patent_dir, output_fp, cpc_fp, sim_spec, n_jobs):

#     model_bert = {
#         "bert": BERTEmbedder(pretrained_model='AI-Growth-Lab/PatentSBERTa')
#     }
#     prep_bert = {
#          "prep-bert": Preprocessor(keep_caps=True, keep_start_section=True, remove_non_alpha=True)
#     }

#     check_files(sim_spec)
#     pretrained_run_models(prep_bert, model_bert, sim_spec, patent_dir, output_fp, cpc_fp)
#     print('Calculated BERT emdeddings')

#### Defining the calculation window

Embeddings are calculated within a time window, which shifts over the dataset and then recalculated.
This procedure is configured with the `SimulationSpecification()`, which has the following attributes:
    
- `year_start`: start year of the entire (sub)set of data to calculate embeddings for.
- `year_end`: id. end year (the end year itself is not included).
- `window_size`: width of the window (in years) to compute embeddings for.
- `window_shift`: number of years between subsequent windows.
- `debug_max_patents`: restrict the number of patents per year (optional; for testing purposes).
    
With the `n_jobs`-parameter you can set the number of concurrent jobs to run. A higher number means faster processing, but be aware that each job takes utilises one CPU-core.

In [4]:
from docembedder.simspec import SimulationSpecification

sim_spec = SimulationSpecification(
    year_start=1976,
    year_end=1980,
    window_size=2,
    window_shift=1,
    cpc_samples_per_patent = 20,
    debug_max_patents = 100,
    # n_patents_per_window = 100,
)

n_jobs=1

#### Computing embeddings

Now that we've defined the window, we can calculate embeddings, using each of the four models.
    
Be aware, depending on the amlount of patents and window size, this will take quite some time, 
and can require a (_very_) large amount of memory. Warnings from the Countvec calculations can be ignored.

All output is stored in a HDF5 file, which contains embeddings for all patents in all windows.

In [5]:
from pathlib import Path

data_path = "./data/USPTO"
input_file = Path(f"{data_path}/brief_text")
year_file = Path(f"{data_path}/input_files/year.csv")
cpc_fp = Path(f"{data_path}/input_files/GPCPCs.txt")
patent_dir = Path("./patents")
output_folder = Path("./output")
output_fp = Path("./output", "patents.h5")
results_fp = Path("./results")

output_folder.mkdir(exist_ok=True)
patent_dir.mkdir(exist_ok=True)


import json
import lzma
import re
from collections import Counter, defaultdict
from pathlib import Path
from typing import List, Union, Dict

from docembedder.models import TfidfEmbedder
from docembedder.preprocessor.preprocessor import Preprocessor
from docembedder.preprocessor.oldprep import OldPreprocessor
from docembedder.models.doc2vec import D2VEmbedder
from docembedder.models import CountVecEmbedder
from docembedder.models import BERTEmbedder

from docembedder.utils import run_models
from docembedder.pretrained_run import pretrained_run_models
import datetime

def check_files(sim_spec):
    for year in range(sim_spec.year_start, sim_spec.year_end):
        if not (patent_dir / f"{year}.xz").is_file():
            raise ValueError(f"Please download patent file {year}.xz and put it in"
                             f"the right directory ({patent_dir})")

# def compute_embeddings_cv(patent_dir, output_fp, cpc_fp, sim_spec, n_jobs):

#     model_cv = {
#         "countvec": CountVecEmbedder(method='sigmoid')
#     }
#     prep_cv = {
#         "prep-countvec": OldPreprocessor(list_path=data_path)
#     }

#     check_files(sim_spec)
#     run_models(prep_cv, model_cv, sim_spec, patent_dir, output_fp, cpc_fp, n_jobs=n_jobs)
#     print('Calculated countvec emdeddings')

    
def compute_embeddings_tfidf(patent_dir, output_fp, cpc_fp, sim_spec, n_jobs):
    
    model_tfidf = {
        "tfidf": TfidfEmbedder(
            ngram_max=1,stop_words='english',stem=False, norm='l1', sublinear_tf=True, min_df=6, max_df=0.665461)
    }
    prep_tfidf = {
        "prep-tfidf": Preprocessor(keep_caps=True, keep_start_section=True, remove_non_alpha=True),
    }

    check_files(sim_spec)
    run_models(prep_tfidf, model_tfidf, sim_spec, patent_dir, output_fp, cpc_fp, n_jobs=n_jobs)


    
    
from docembedder.simspec import SimulationSpecification

sim_spec = SimulationSpecification(
    year_start=1976,
    year_end=1982,
    window_size=3,
    window_shift=1,
    cpc_samples_per_patent = 10,
    debug_max_patents = 100,
    # n_patents_per_window = 100,
)

n_jobs=1    


args={'patent_dir': patent_dir, 'output_fp': output_fp, 'cpc_fp': cpc_fp, 'sim_spec': sim_spec, 'n_jobs': n_jobs}

# # Countvec
# compute_embeddings_cv(**args)

# Tf-Idf
compute_embeddings_tfidf(**args)

# # Doc2Vec
# compute_embeddings_doc2vec(**args)

# # BERT
# compute_embeddings_bert(**args)

 25%|███████████                                 | 1/4 [02:15<06:47, 135.96s/it]


KeyboardInterrupt: 

## 3. Impact and novelty scores

### 3.1. Calculating the scores

After we've computed and stored the embeddings, we compute novelty and impact scores. The result is a dictionary per model, each containing the novelties and impacts for each patent.


_Note on exponents_

The exponents (`[1.0, 2.0, 3.0]`) are used in the calculations to reward patents that are more similar to the patent under consideration. The backward and forward similarities for each patent is calculated based on the mean of all cosine similarities with the preceding and following patents in the window, using the formula `(x1**a + x2**a + ...)**(1/a)`, with `a` being the exponent. An `a` larger than 1 increases the weight of similarities closer to 1, i.e. of embeddings that are more similar to the one under consideration. The output includes the result for each exponent.

In [5]:
from docembedder.analysis import DocAnalysis
from docembedder.datamodel import DataModel
from collections import defaultdict

import pandas as pd

def compute_impacts(embedding_fp, output_dir):
    exponents = [1.0, 2.0, 3.0]

    impact_novel = defaultdict(lambda: defaultdict(list))

    with DataModel(embedding_fp, read_only=False) as data:
        analysis = DocAnalysis(data)
       
        for window, model in data.iterate_window_models():
            print(f'window: {window}')
            print(f'model: {model}')
            results = analysis.impact_novelty_results(window, model, exponents, cache=False, n_jobs=1)

            for expon, res in results.items():
                if expon == exponents[0]:
                    impact_novel[model]["patent_ids"].extend(res["patent_ids"])
                impact_novel[model][f"impact-{expon}"].extend(res["impact"])
                impact_novel[model][f"novelty-{expon}"].extend(res["novelty"])

    output_dir.mkdir(exist_ok=True, parents=True)

    for model, data in impact_novel.items():
        classifier_name = model.split("-")[-1]
        impact_fp = Path(output_dir, f"impact-{classifier_name}.csv")
        pd.DataFrame(impact_novel[model]).sort_values("patent_ids").to_csv(impact_fp, index=False)


compute_impacts(embedding_fp=output_fp, output_dir=results_fp)

window: 1976-1978
model: prep-tfidf-tfidf
window: 1977-1979
model: prep-tfidf-tfidf
window: 1978-1980
model: prep-tfidf-tfidf
window: 1979-1981
model: prep-tfidf-tfidf


### 3.2. Output

After the computations are done, novelty and impact scores are written to CSV-files in the results folder. One file per model, with novelty and impact scores for each exponent. The key column refers back to the patent ID's from the original data.

Below is a list of the resulting files.

In [4]:
[str(path.absolute()) for path in results_fp.iterdir()]

['/Users/a6159737/Documents/Utrecht University/PhD/Projects/DS/patents/results/impact-tfidf.csv']